# Text Classification
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. Please check the pdf file for more details.*

In this exercise you will:
    
- implement a of spam classifier with **Naive Bayes method** for real world email messages
- learn the **training and testing phase** for Naive Bayes classifier  
- get an idea of the **precision-recall** tradeoff

In [1]:
# some basic imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
# ham_train contains the occurrences of each word in ham emails. 1-by-N vector
ham_train = np.loadtxt('ham_train.csv', delimiter=',')
# spam_train contains the occurrences of each word in spam emails. 1-by-N vector
spam_train = np.loadtxt('spam_train.csv', delimiter=',')
# N is the size of vocabulary.
N = ham_train.shape[0]
# There 9034 ham emails and 3372 spam emails in the training samples
num_ham_train = 9034
num_spam_train = 3372
# Do smoothing
x = np.vstack([ham_train, spam_train]) + 1

# ham_test contains the occurences of each word in each ham test email. P-by-N vector, with P is number of ham test emails.
i,j,ham_test = np.loadtxt('ham_test.txt').T
i = i.astype(np.int)
j = j.astype(np.int)
ham_test_tight = scipy.sparse.coo_matrix((ham_test, (i - 1, j - 1)))
ham_test = scipy.sparse.csr_matrix((ham_test_tight.shape[0], ham_train.shape[0]))
ham_test[:, 0:ham_test_tight.shape[1]] = ham_test_tight
# spam_test contains the occurences of each word in each spam test email. Q-by-N vector, with Q is number of spam test emails.
i,j,spam_test = np.loadtxt('spam_test.txt').T
i = i.astype(np.int)
j = j.astype(np.int)
spam_test_tight = scipy.sparse.csr_matrix((spam_test, (i - 1, j - 1)))
spam_test = scipy.sparse.csr_matrix((spam_test_tight.shape[0], spam_train.shape[0]))
spam_test[:, 0:spam_test_tight.shape[1]] = spam_test_tight


D:\program\Anaconda3\lib\site-packages\scipy\sparse\_index.py:118: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


## Now let's implement a ham/spam email classifier. Please refer to the PDF file for details

In [3]:
from likelihood import likelihood
# TODO
# Implement a ham/spam email classifier, and calculate the accuracy of your classifier

# Hint: you can directly do matrix multiply between scipy.sparse.coo_matrix and numpy.array.
# Specifically, you can use sparse_matrix * np_array to do this. Note that when you use "*" operator
# between numpy array, this is typically an elementwise multiply.

# begin answer
l = likelihood(x)
np.min(l)

8.54065050718653e-07

In [4]:
# List the top 10 words that are most indicative of the SPAM class
ratio = l[1] / l[0]
sorted_ratio_index = np.argsort(-ratio) # sort ratio 
top10_ratio_index = sorted_ratio_index[:10] # get top 10 words' index
print(top10_ratio_index)
f = open("all_word_map.txt", 'r') 
words_map = f.readlines()
words = [word.split('\t')[0] for word in words_map] # get all words
for i in top10_ratio_index:
    # print top 10 words 
    print(words[i])
#     print(ham_train[i], spam_train[i]) 

[30032 75525 38175 45152  9493 65397 37567 13612 56929 19956]
nbsp
viagra
pills
cialis
voip
php
meds
computron
sex
width


In [5]:
spam_size = spam_test.shape[0]
ham_size = ham_test.shape[0]

In [6]:
p_spam = 1.0 * num_spam_train / (num_ham_train + num_spam_train)
p_ham = 1 - p_spam

In [7]:
# use logarithm to avoid floating underflow problem
l = np.log(l)
p_ham = np.log(p_ham)
p_spam = np.log(p_spam)

In [8]:
postp_ham = np.zeros((ham_size, 2))
for i in range(ham_test.shape[0]):
    postp_ham[i, 0] = ham_test[i, :].dot(l[0].T) + p_ham # ham test's posterior probability after logarithm of ham ignoring P(x)
    postp_ham[i, 1] = ham_test[i, :].dot(l[1].T) + p_spam # ham test's posterior probability after logarithm of spam ignoring P(x)
postp_spam = np.zeros((spam_size, 2))
for i in range(spam_test.shape[0]):
    postp_spam[i, 0] = spam_test[i, :].dot(l[0].T) + p_ham # spam test's posterior probability after logarithm of ham ignoring P(x)
    postp_spam[i, 1] = spam_test[i, :].dot(l[1].T) + p_spam # spam test's posterior probability after logarithm of spam ignoring P(x)

In [10]:
# predict spam_test. 0 for ham, 1 for spam
pred_spam_test = postp_spam[:, 0] < postp_spam[:, 1]
# predict ham_test. 0 for ham, 1 for spam
pred_ham_test = postp_ham[:, 0] < postp_ham[:, 1]

In [11]:
# confusion matrix
TP = sum(pred_spam_test)
FN = spam_size - TP
FP = sum(pred_ham_test)
TN = ham_size - FP
print("TP: {}, FP: {}".format(TP, FP))
print("FN: {}, TN: {}".format(FN, TN))

TP: 1073, FP: 16
FN: 51, TN: 2995


In [12]:
# accuracy of spam filter on the testing set
acc = (TP + TN) / (spam_size + ham_size)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print("accuracy: {:.2f}%".format(acc * 100))
print("precision: {:.2f}%".format(precision * 100))
print("recall: {:.2f}%".format(recall * 100))

#end answer

accuracy: 98.38%
precision: 98.53%
recall: 95.46%
